Dataset GT 개수 분포 확인

In [6]:
# Cell 1: 기본 설정 및 split 파일 로드
import os
from collections import defaultdict

# —————————————————————————————————————————————————————————————
# 여러분 환경에 맞게 경로만 수정하세요
label_dir    = 'datasets/kaist-rgbt/train/labels'
train_all    = 'datasets/kaist-rgbt/train-all-04.txt'
train_split  = 'datasets/kaist-rgbt/train-split-bymanual.txt'
val_split    = 'datasets/kaist-rgbt/val-split-bymanual.txt'
# —————————————————————————————————————————————————————————————

def load_split(file_path):
    """split 파일에서 비어 있지 않은 줄만 리스트로 반환"""
    with open(file_path, 'r') as f:
        return [line.strip() for line in f if line.strip()]

train_all_list = load_split(train_all)
train_list = load_split(train_split)
val_list   = load_split(val_split)

print(f"All train images: {len(train_all_list)}개")
print(f"Train images: {len(train_list)}개,  Val images: {len(val_list)}개")


All train images: 12538개
Train images: 9605개,  Val images: 2933개


In [7]:
# Cell 2: GT 개수 집계 함수 정의 및 실행
def count_gt_per_group(file_list):
    """
    file_list: 이미지 파일 경로 리스트
    return: { 'setXX_VYYY': GT개수, … }
    """
    counts = defaultdict(int)
    for img_path in file_list:
        # 파일명에서 basename 추출 (확장자 제거)
        base = os.path.splitext(os.path.basename(img_path))[0]  # ex: set00_V000_I00003
        group = '_'.join(base.split('_')[:2])                  # ex: set00_V000

        # 대응하는 라벨 파일 경로
        lbl = os.path.join(label_dir, base + '.txt')
        if not os.path.isfile(lbl):
            # 파일이 없으면 경고만 출력
            print(f"Warning: label not found → {lbl}")
            continue

        # 라벨 파일의 유효한 줄 수 = GT box 개수
        with open(lbl, 'r') as f:
            n = sum(1 for line in f if line.strip())
        counts[group] += n

    return counts

train_all_counts = count_gt_per_group(train_all_list)
# train_counts = count_gt_per_group(train_list)
# val_counts   = count_gt_per_group(val_list)


In [8]:
# Cell 3: 집계 결과 출력
print("▶ Train split GT counts by V-group:")
for grp in sorted(train_all_counts):
    print(f"  {grp}: GT {train_all_counts[grp]}개")

# print("▶ Train split GT counts by V-group:")
# for grp in sorted(train_counts):
#     print(f"  {grp}: GT {train_counts[grp]}개")

# print("\n▶ Val split GT counts by V-group:")
# for grp in sorted(val_counts):
#     print(f"  {grp}: GT {val_counts[grp]}개")


▶ Train split GT counts by V-group:
  set00_V000: GT 259개
  set00_V001: GT 285개
  set00_V002: GT 119개
  set00_V003: GT 566개
  set00_V004: GT 310개
  set00_V005: GT 117개
  set00_V006: GT 180개
  set00_V007: GT 312개
  set00_V008: GT 599개
  set01_V000: GT 642개
  set01_V001: GT 820개
  set01_V002: GT 105개
  set01_V003: GT 399개
  set01_V004: GT 126개
  set01_V005: GT 48개
  set02_V000: GT 466개
  set02_V001: GT 923개
  set02_V002: GT 666개
  set02_V003: GT 457개
  set02_V004: GT 361개
  set03_V000: GT 793개
  set03_V001: GT 1057개
  set04_V000: GT 1251개
  set04_V001: GT 1288개
  set05_V000: GT 1163개


In [10]:
# 특성 부여
TIME_OF_DAY = {
    'Day': {'set00', 'set01', 'set02', 'set06', 'set07', 'set08'},
    'Night': {'set03', 'set04', 'set05', 'set09', 'set10', 'set11'},
}
REGION = {
    'Campus': {'set00', 'set03'},
    'Road': {'set01', 'set04'},
    'Downtown': {'set02', 'set05'},
}

# 코드 추가: train/val set 최적조합 찾아내기
# 조건:
# - 전체 데이터 개수비 train:val = 8:2에서 7:3
# - TIME_OF_DAY 균등 분포 (train, val set 내에 Day:Night 비율 동일)
# - REGION 균등 분포 (train, val set 내에 Campus:Road:Downtown 비율 동일)


In [19]:
from itertools import combinations
from collections import Counter

# 모든 V-group 목록
v_groups = sorted(train_all_counts.keys())

# 각 V-group이 속한 특성 매핑
def get_group_feature(v_group):
    set_id = v_group.split('_')[0]
    time = 'Day' if set_id in TIME_OF_DAY['Day'] else 'Night'
    region = [k for k, v in REGION.items() if set_id in v][0]

    # 이미지 수는 파일 목록에서 직접 카운트
    img_count = sum(1 for p in train_all_list if f"{v_group}_" in os.path.basename(p))
    gt_count = train_all_counts.get(v_group, 0)

    return {
        'time': time,
        'region': region,
        'img_count': img_count,
        'gt_count': gt_count
    }

group_features = {vg: get_group_feature(vg) for vg in v_groups}

print(group_features)

{'set00_V000': {'time': 'Day', 'region': 'Campus', 'img_count': 561, 'gt_count': 259}, 'set00_V001': {'time': 'Day', 'region': 'Campus', 'img_count': 699, 'gt_count': 285}, 'set00_V002': {'time': 'Day', 'region': 'Campus', 'img_count': 627, 'gt_count': 119}, 'set00_V003': {'time': 'Day', 'region': 'Campus', 'img_count': 408, 'gt_count': 566}, 'set00_V004': {'time': 'Day', 'region': 'Campus', 'img_count': 520, 'gt_count': 310}, 'set00_V005': {'time': 'Day', 'region': 'Campus', 'img_count': 383, 'gt_count': 117}, 'set00_V006': {'time': 'Day', 'region': 'Campus', 'img_count': 274, 'gt_count': 180}, 'set00_V007': {'time': 'Day', 'region': 'Campus', 'img_count': 399, 'gt_count': 312}, 'set00_V008': {'time': 'Day', 'region': 'Campus', 'img_count': 500, 'gt_count': 599}, 'set01_V000': {'time': 'Day', 'region': 'Road', 'img_count': 563, 'gt_count': 642}, 'set01_V001': {'time': 'Day', 'region': 'Road', 'img_count': 289, 'gt_count': 820}, 'set01_V002': {'time': 'Day', 'region': 'Road', 'img_coun

In [ ]:
from itertools import combinations
from collections import defaultdict, Counter

# 기준 비율
MIN_RATIO = 0.68
MAX_RATIO = 0.72

# 키별 비율 차이의 총합을 반환
def ratio_diff(counter1, counter2):
    keys = set(counter1.keys()).union(counter2.keys())
    total1 = sum(counter1.values())
    total2 = sum(counter2.values())
    diff_sum = 0.0
    for k in keys:
        r1 = counter1[k] / total1 if total1 else 0
        r2 = counter2[k] / total2 if total2 else 0
        diff_sum += abs(r1 - r2)
    return diff_sum

# 그룹별 통계 계산
def extract_counts(groups):
    total_img = total_gt = 0
    time_counter = Counter()
    region_counter = Counter()
    for g in groups:
        f = group_features[g]
        total_img += f['img_count']
        total_gt += f['gt_count']
        time_counter[f['time']] += f['img_count']
        region_counter[f['region']] += f['img_count']
    return total_img, total_gt, time_counter, region_counter

# V-group 목록 및 특성 구성
v_groups = sorted(set(os.path.splitext(os.path.basename(p))[0].rsplit('_', 1)[0] for p in train_all_list))
group_features = {vg: get_group_feature(vg) for vg in v_groups}

# 최적 split 탐색
best_diff = float('inf')
best_split = None

for n in range(int(len(v_groups) * MIN_RATIO), int(len(v_groups) * MAX_RATIO) + 1):
    for train_subset in combinations(v_groups, n):
        val_subset = [vg for vg in v_groups if vg not in train_subset]

        train_img, train_gt, train_time, train_region = extract_counts(train_subset)
        val_img, val_gt, val_time, val_region = extract_counts(val_subset)

        # 조건 1: 전체 이미지 수 기준 비율 확인
        ratio = train_img / (train_img + val_img)
        if not (MIN_RATIO <= ratio <= MAX_RATIO):
            continue

        # 조건 2: 이미지:GT 비율 유사성
        train_ratio = train_gt / train_img if train_img else 0
        val_ratio = val_gt / val_img if val_img else 0
        diff_ratio = abs(train_ratio - val_ratio)
        if diff_ratio > 0.2:  # 허용 오차 완화
            continue

        # 조건 3: 낮/밤 비율
        time_diff = ratio_diff(train_time, val_time)

        # 조건 4: region 비율
        region_diff = ratio_diff(train_region, val_region)

        # 총합 오차 계산
        total_diff = diff_ratio + time_diff + region_diff

        if total_diff < best_diff:
            best_diff = total_diff
            best_split = (set(train_subset), set(val_subset))


✅ 최적 분할 결과:
Train (17 groups): ['set00_V000', 'set00_V002', 'set00_V003', 'set00_V004', 'set00_V007', 'set00_V008', 'set01_V000', 'set01_V001', 'set01_V002', 'set01_V003', 'set01_V005', 'set02_V001', 'set02_V003', 'set02_V004', 'set03_V000', 'set04_V000', 'set05_V000']
Val   (8 groups): ['set00_V001', 'set00_V005', 'set00_V006', 'set01_V004', 'set02_V000', 'set02_V002', 'set03_V001', 'set04_V001']

[Train]
 - 이미지 수: 8578, GT 수: 9127, 비율: 1.064
 - 낮/밤 분포: {'Day': 5749, 'Night': 2829}
 - 지역 분포: {'Campus': 4115, 'Downtown': 1870, 'Road': 2593}

[Val]
 - 이미지 수: 3960, GT 수: 4185, 비율: 1.057
 - 낮/밤 분포: {'Day': 2593, 'Night': 1367}
 - 지역 분포: {'Campus': 1923, 'Road': 1212, 'Downtown': 825}


In [26]:
# 결과 확인
if best_split is None:
    print("❌ 조건을 만족하는 분할을 찾지 못했습니다.")
else:
    train_final, val_final = best_split
    print("✅ 최적 분할 결과:")
    print(f"Train ({len(train_final)} groups): {sorted(train_final)}")
    print(f"Val   ({len(val_final)} groups): {sorted(val_final)}")

    # 통계 요약 함수
    def summary(name, subset):
        img_sum, gt_sum, time_cnt, region_cnt = extract_counts(subset)

        # 시간대/지역별 GT 카운트 따로 계산
        time_gt_counter = Counter()
        region_gt_counter = Counter()
        for g in subset:
            f = group_features[g]
            time_gt_counter[f['time']] += f['gt_count']
            region_gt_counter[f['region']] += f['gt_count']

        print(f"\n[{name}]")
        print(f" - 전체 이미지 수: {img_sum}, 전체 GT 수: {gt_sum}, 이미지당 평균 GT: {gt_sum/img_sum:.3f}")

        print(" - 낮/밤 분포:")
        for k in sorted(time_cnt):
            img_ratio = time_cnt[k] / img_sum if img_sum else 0
            gt = time_gt_counter[k]
            gt_ratio = gt / gt_sum if gt_sum else 0
            print(f"    {k}: {time_cnt[k]}장 ({img_ratio:.2%}), GT {gt}개 ({gt_ratio:.2%})")

        print(" - 지역 분포:")
        for k in sorted(region_cnt):
            img_ratio = region_cnt[k] / img_sum if img_sum else 0
            gt = region_gt_counter[k]
            gt_ratio = gt / gt_sum if gt_sum else 0
            print(f"    {k}: {region_cnt[k]}장 ({img_ratio:.2%}), GT {gt}개 ({gt_ratio:.2%})")


    summary("Train", train_final)
    summary("Val", val_final)


✅ 최적 분할 결과:
Train (17 groups): ['set00_V000', 'set00_V002', 'set00_V003', 'set00_V004', 'set00_V007', 'set00_V008', 'set01_V000', 'set01_V001', 'set01_V002', 'set01_V003', 'set01_V005', 'set02_V001', 'set02_V003', 'set02_V004', 'set03_V000', 'set04_V000', 'set05_V000']
Val   (8 groups): ['set00_V001', 'set00_V005', 'set00_V006', 'set01_V004', 'set02_V000', 'set02_V002', 'set03_V001', 'set04_V001']

[Train]
 - 전체 이미지 수: 8578, 전체 GT 수: 9127, 이미지당 평균 GT: 1.064
 - 낮/밤 분포:
    Day: 5749장 (67.02%), GT 5920개 (64.86%)
    Night: 2829장 (32.98%), GT 3207개 (35.14%)
 - 지역 분포:
    Campus: 4115장 (47.97%), GT 2958개 (32.41%)
    Downtown: 1870장 (21.80%), GT 2904개 (31.82%)
    Road: 2593장 (30.23%), GT 3265개 (35.77%)

[Val]
 - 전체 이미지 수: 3960, 전체 GT 수: 4185, 이미지당 평균 GT: 1.057
 - 낮/밤 분포:
    Day: 2593장 (65.48%), GT 1840개 (43.97%)
    Night: 1367장 (34.52%), GT 2345개 (56.03%)
 - 지역 분포:
    Campus: 1923장 (48.56%), GT 1639개 (39.16%)
    Downtown: 825장 (20.83%), GT 1132개 (27.05%)
    Road: 1212장 (30.61%), GT 1

In [17]:
# 전체 GT 수
total_gt = sum(train_all_counts.values())

# 비율 기준
MIN_RATIO = 0.68  # 약 7:3
MAX_RATIO = 0.82  # 약 8:2

# 기준 비율을 V-group 수 기준으로 계산
def compute_ratios(vg_list):
    feats = [group_features[vg] for vg in vg_list]
    time_cnt = Counter(t for t, _ in feats)
    region_cnt = Counter(r for _, r in feats)
    return time_cnt, region_cnt

# 기준 비율 (전체 비율)
total_time, total_region = compute_ratios(v_groups)
print(total_time)
print(total_region)

Counter({'Day': 20, 'Night': 5})
Counter({'Campus': 11, 'Road': 8, 'Downtown': 6})
